In [1]:
'''بِسْمِ اللهِ الرَّحْمٰنِ الرَّحِيْمِ'''

'بِسْمِ اللهِ الرَّحْمٰنِ الرَّحِيْمِ'

In [2]:
'''سُبْحَـٰنَكَ لَا عِلْمَ لَنَآ إِلَّا مَا عَلَّمْتَنَآ ۖ إِنَّكَ أَنتَ الْعَلِيمُ الْحَكِيمُ'''

'سُبْحَـٰنَكَ لَا عِلْمَ لَنَآ إِلَّا مَا عَلَّمْتَنَآ ۖ إِنَّكَ أَنتَ الْعَلِيمُ الْحَكِيمُ'

In [3]:
from IPython.core.display import HTML
HTML(r"""
<style>
    * {
        font-family: Monaco;
    }
</style>
""")

In [4]:
import os,sys,warnings,re,math,gc,time
warnings.filterwarnings("ignore")
os.cpu_count()

12

In [5]:
os.environ["POLARS_MAX_THREADS"] = "40"
import numpy as np
import pandas as pd
import polars as pl
import polars.selectors as cs
from glob import glob
from pathlib import Path
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
pio.templates["mod"] = go.layout.Template(layout=dict(font=dict(family="Monaco",size=15)))
pio.templates.default = "mod+plotly_dark"
from sklearn.metrics import auc,roc_auc_score
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import miceforest as mf
from miceforest import mean_match_shap,mean_match_default,mean_match_fast_cat
%xmode Minimal

Exception reporting mode: Minimal


In [6]:
pl.Config.set_fmt_str_lengths(100)
pl.Config.set_float_precision(2)
pl.Config.set_fmt_float("full")
pl.Config.set_tbl_rows(40)

polars.config.Config

In [7]:
path_to_train = Path("Downloads/credit_risk/train/")
path_to_test = Path("Downloads/credit_risk/test/")
path_to_features = Path("Downloads/feature_definitions.csv")

In [8]:
feat_df = pl.read_csv(path_to_features)
feat_df.head()

Variable,Description
str,str
"""actualdpd_943P""","""Days Past Due (DPD) of previous contract (actual)."""
"""actualdpdtolerance_344P""","""DPD of client with tolerance."""
"""addres_district_368M""","""District of the person's address."""
"""addres_role_871L""","""Role of person's address."""
"""addres_zip_823M""","""Zip code of the address."""


In [9]:
all_train_files = glob("Downloads/credit_risk/train/*.parquet")
all_test_files = glob("Downloads/credit_risk/test/*.parquet")
train_files_df = pl.DataFrame({"index":range(len(all_train_files)),"path":all_train_files})
test_files_df = pl.DataFrame({"index":range(len(all_test_files)),"path":all_test_files})
train_files_df = (
    train_files_df
    .with_columns(
        (pl.col("path").str.split("/").list.get(-1)).alias("filename")
    )
    .sort(by="filename")
)
test_files_df = (
    test_files_df
    .with_columns(
        (pl.col("path").str.split("/").list.get(-1)).alias("filename")
    )
    .sort(by="filename")
)
display(train_files_df.head())
display(test_files_df.head())

index,path,filename
i64,str,str
31,"""Downloads/credit_risk/train/train_applprev_1_0.parquet""","""train_applprev_1_0.parquet"""
13,"""Downloads/credit_risk/train/train_applprev_1_1.parquet""","""train_applprev_1_1.parquet"""
0,"""Downloads/credit_risk/train/train_applprev_2.parquet""","""train_applprev_2.parquet"""
7,"""Downloads/credit_risk/train/train_base.parquet""","""train_base.parquet"""
11,"""Downloads/credit_risk/train/train_credit_bureau_a_1_0.parquet""","""train_credit_bureau_a_1_0.parquet"""


index,path,filename
i64,str,str
31,"""Downloads/credit_risk/test/test_applprev_1_0.parquet""","""test_applprev_1_0.parquet"""
35,"""Downloads/credit_risk/test/test_applprev_1_1.parquet""","""test_applprev_1_1.parquet"""
9,"""Downloads/credit_risk/test/test_applprev_1_2.parquet""","""test_applprev_1_2.parquet"""
1,"""Downloads/credit_risk/test/test_applprev_2.parquet""","""test_applprev_2.parquet"""
33,"""Downloads/credit_risk/test/test_base.parquet""","""test_base.parquet"""


In [10]:
train_files_df.filter(pl.col("filename").str.contains("base"))

index,path,filename
i64,str,str
7,"""Downloads/credit_risk/train/train_base.parquet""","""train_base.parquet"""


In [11]:
train_base = (
    pl.scan_parquet(all_train_files[7])
    .select(
        pl.col("case_id").cast(pl.UInt32).alias("case_id_base"),
        cs.contains("date").str.to_date().alias("date"),
        pl.col("target").cast(pl.UInt8)
    )
    .with_columns(
        pl.col("date").dt.month().alias("month"),
        pl.col("date").dt.week().alias("week"),
        pl.col("date").dt.weekday().alias("weekday")
    )

    .select(~cs.contains(["target","date"]),cs.contains("target"))
)
display(train_base.fetch().head())
train_base.collect(streaming=True).shape

case_id_base,month,week,weekday,target
u32,i8,i8,i8,u8
0,1,1,4,0
1,1,1,4,0
2,1,1,5,0
3,1,1,4,0
4,1,1,5,1


(1526659, 5)

In [12]:
test_files_df.filter(pl.col("filename").str.contains("base"))

index,path,filename
i64,str,str
33,"""Downloads/credit_risk/test/test_base.parquet""","""test_base.parquet"""


In [13]:
test_base = (
    pl.scan_parquet(all_test_files[33])
    .select(
        pl.col("case_id").cast(pl.UInt32).alias("case_id_base"),
        cs.contains("date").str.to_date().alias("date"),
    )
    .with_columns(
        pl.col("date").dt.month().alias("month"),
        pl.col("date").dt.week().alias("week"),
        pl.col("date").dt.weekday().alias("weekday")
    )

    .select(~cs.contains(["date"]),cs.contains("target"))
)
display(test_base.fetch().head())
print(test_base.collect(streaming=True).shape)

case_id_base,month,week,weekday
u32,i8,i8,i8
57543,10,41,2
57549,10,41,2
57551,10,41,2
57552,10,41,3
57569,10,41,2


(10, 4)


In [14]:
train_files_df.filter(pl.col("filename").str.contains("appl"))

index,path,filename
i64,str,str
31,"""Downloads/credit_risk/train/train_applprev_1_0.parquet""","""train_applprev_1_0.parquet"""
13,"""Downloads/credit_risk/train/train_applprev_1_1.parquet""","""train_applprev_1_1.parquet"""
0,"""Downloads/credit_risk/train/train_applprev_2.parquet""","""train_applprev_2.parquet"""


## Functions

In [15]:
def select_imputable_cols(df: pl.LazyFrame) -> pl.LazyFrame:
    col_list = (
        df
        .collect(streaming=True)
        .pipe(lambda df: (df.null_count()/df.shape[0])*100)
        .transpose(include_header=True,column_names=["percentage of nulls"])
        .join(
            feat_df,
            left_on="column",
            right_on="Variable",
            how="left"
        )
        .sort(by="percentage of nulls")
        .select(["column","Description","percentage of nulls"])
        .filter(pl.col("percentage of nulls") < 10)
        ["column"].to_list()
    )
    _ = df.select(col_list).lazy()
    gc.collect()
    return _

In [30]:
train_previous_application = (
    pl.scan_parquet(path_to_train/"train_applprev_1_*.parquet")
    .drop(cs.contains("date"))
    .select(
        pl.col("case_id").cast(pl.UInt32),
        pl.col("num_group1").cast(pl.UInt8),
        cs.float().cast(pl.Float32),
        cs.string().cast(pl.Categorical),
    )
    # .filter(pl.col("num_group1") == 0)
    .pipe(select_imputable_cols)
    .fill_null(strategy="mean")
)
train_previous_application.fetch().head(10)

case_id,num_group1,cancelreason_3545846M,district_544M,education_1138M,postype_4733339M,profession_152M,rejectreason_755M,rejectreasonclient_4145042M,status_219L,actualdpd_943P,mainoccupationinc_437A,credacc_credlmt_575A,credamount_590A,downpmt_134A,credtype_587L,inittransactioncode_279L,annuity_853A,pmtnum_8L,tenor_203L
u32,u8,cat,cat,cat,cat,cat,cat,cat,cat,f32,f32,f32,f32,f32,cat,cat,f32,f32,f32
2,0,"""a55475b1""","""P136_108_173""","""P97_36_170""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""D""",0.00,8200.00,0.00,10000.00,0.00,"""CAL""","""CASH""",640.20,24.00,24.00
2,1,"""a55475b1""","""P136_108_173""","""P97_36_170""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""D""",0.00,8200.00,0.00,16000.00,0.00,"""CAL""","""CASH""",1682.40,12.00,12.00
3,0,"""P94_109_143""","""P131_33_167""","""P97_36_170""","""a55475b1""","""a55475b1""","""P94_109_143""","""a55475b1""","""D""",0.00,11000.00,0.00,59999.80,0.00,"""CAL""","""CASH""",6140.00,12.00,12.00
4,0,"""P24_27_36""","""P194_82_174""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""T""",0.00,16000.00,0.00,40000.00,0.00,"""CAL""","""CASH""",2556.60,24.00,24.00
5,0,"""P85_114_140""","""P54_133_26""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""T""",0.00,62000.00,2161.10,45335.69,471.80,"""CAL""","""CASH""",3745.33,18.44,18.44
6,0,"""P94_109_143""","""P82_154_182""","""a55475b1""","""a55475b1""","""a55475b1""","""P94_109_143""","""a55475b1""","""D""",0.00,37000.00,0.00,15980.00,0.00,"""CAL""","""CASH""",1773.80,11.00,11.00
6,1,"""P94_109_143""","""P82_154_182""","""P97_36_170""","""a55475b1""","""a55475b1""","""P94_109_143""","""a55475b1""","""D""",0.00,35000.00,0.00,32000.00,0.00,"""CAL""","""CASH""",4189.60,11.00,11.00
6,2,"""a55475b1""","""a55475b1""","""P97_36_170""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""D""",0.00,14000.00,0.00,17380.00,0.00,"""COL""","""POS""",1110.40,24.00,24.00
10,0,"""P73_130_169""","""P38_65_49""","""P17_36_170""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""T""",0.00,57000.00,0.00,100000.00,0.00,"""CAL""","""CASH""",10916.60,12.00,12.00


In [17]:
train_files_df.filter(pl.col("filename").str.contains("applprev_2"))

index,path,filename
i64,str,str
0,"""Downloads/credit_risk/train/train_applprev_2.parquet""","""train_applprev_2.parquet"""


In [23]:
train_previous_application_status = (
    pl.scan_parquet(all_train_files[0])
    .select(
        pl.col("case_id").cast(pl.UInt32),
        pl.col("num_group1").cast(pl.UInt8),
        pl.col("num_group2").cast(pl.UInt8),
        cs.string().cast(pl.Categorical)
    )
)
train_previous_application_status.fetch().head()

case_id,num_group1,num_group2,cacccardblochreas_147M
u32,u8,u8,cat
2,1,1,null
2,0,1,null
2,0,0,null
2,1,0,null
3,0,1,null


In [31]:
(
    train_previous_application
    .select("num_group1")
    .unique()
    .collect(streaming=True)
    .sort(by="num_group1")
)

num_group1
u8
0
1
2
3
4
5
6
7
8


In [27]:
(
    train_previous_application_status
    .select("num_group1")
    .unique()
    .collect(streaming=True)
    .sort(by="num_group1")
)

num_group1
u8
0
1
2
3
4
5
6
7
8


In [28]:
(
    train_previous_application_status
    .select("num_group2")
    .unique()
    .collect(streaming=True)
    .sort(by="num_group2")
)

num_group2
u8
0
1
2
3
4
5
6
7
8
